# Test

In [2]:
from transformers import pipeline

c:\Users\river\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\river\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\river\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\river\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [ ]:
fill = pipeline('fill-mask', model='EgyBERTa', tokenizer='EgyBERTa')

In [ ]:
# دا مجرد رأى و اله اعلم و ابقا
# masking word (اله)
fill("كيف نترك <mask> بلا تخفيف")

[{'score': 0.016952363774180412,
  'token': 302,
  'token_str': ' من',
  'sequence': 'كيف نترك من بلا تخفيف'},
 {'score': 0.014255023561418056,
  'token': 225,
  'token_str': ' ',
  'sequence': 'كيف نترك  بلا تخفيف'},
 {'score': 0.013390934094786644,
  'token': 275,
  'token_str': ' و',
  'sequence': 'كيف نترك و بلا تخفيف'},
 {'score': 0.008837600238621235,
  'token': 320,
  'token_str': ' ان',
  'sequence': 'كيف نترك ان بلا تخفيف'},
 {'score': 0.008807444013655186,
  'token': 333,
  'token_str': ' فى',
  'sequence': 'كيف نترك فى بلا تخفيف'}]

# Get contextual word embedding 

In [6]:
import numpy as np
import torch
from transformers import RobertaTokenizer, AutoModel,RobertaForMaskedLM,AutoTokenizer


def get_word_idx(sent: str, word: str):
    return sent.split(" ").index(word)


def get_hidden_states(encoded, token_ids_word, model, layers):
    """Push input IDs through model. Stack and sum `layers` (last four by default).
    Select only those subword token outputs that belong to our word of interest
    and average them."""
    with torch.no_grad():
        output = model(**encoded)

    # Get all hidden states
    states = output.hidden_states
    # Stack and sum all requested layers
    output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
    # Only select the tokens that constitute the requested word
    word_tokens_output = output[token_ids_word]

    return word_tokens_output.mean(dim=0)


def get_word_vector(sent, idx, tokenizer, model, layers):
    """Get a word vector by first tokenizing the input sentence, getting all token idxs
    that make up the word of interest, and then `get_hidden_states`."""
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
    # get all token idxs that belong to the word of interest
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)

    return get_hidden_states(encoded, token_ids_word, model, layers)


# Test word similarity with cosine similarity sklearn

In [7]:
# Use last four layers by default

layers = [-4, -3, -2, -1] 
tokenizer = AutoTokenizer.from_pretrained('EgyBERTa', max_len=512)
model = AutoModel.from_pretrained("EgyBERTa", output_hidden_states=True)

sent = "الحباه حلوه" 
idx = get_word_idx(sent, "حلوه")

word_embedding = get_word_vector(sent, idx, tokenizer, model, layers)
word_embedding 

Some weights of the model checkpoint at EgyBERTa were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at EgyBERTa and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([ 2.0237e-01, -1.9478e+00, -3.0412e+00, -3.7727e+00, -3.3422e+00,
        -6.5008e+00,  1.1251e+00, -9.0984e-01, -4.3397e+00, -6.8665e+00,
        -1.6424e+00, -4.9605e+00, -5.0162e+00, -1.7020e-01, -5.9561e+00,
        -1.4974e+00, -4.5487e+00, -4.7444e-01,  2.4535e+00,  9.6574e-01,
        -1.2342e+00,  6.5007e+00, -2.0057e+00,  1.6802e+00, -3.4554e+00,
         3.4686e+00, -2.8246e+00,  6.7783e+00,  1.4221e+00, -1.7852e+00,
         1.0039e+01,  4.6123e+00, -3.9829e+00, -3.0084e+00, -1.0839e+00,
         3.7969e+00, -2.8679e+00,  6.8511e+00, -6.1027e+00, -1.7925e+00,
        -5.8851e-01, -3.5449e+00, -2.0798e-01,  6.2225e+00, -1.4809e+00,
         4.7526e-02,  4.1646e+00, -3.8881e+00,  2.6679e+00,  4.3125e+00,
        -7.2887e+00, -1.9245e+00,  8.2278e-01, -2.3780e+00,  4.0661e+00,
        -4.0736e+00, -5.1413e-01, -2.7918e+00, -6.5185e+00, -2.9290e+00,
         1.0714e+00,  6.2348e+00,  2.4318e+00,  7.6076e-01, -4.1703e+00,
        -3.4822e+00, -8.1879e+00, -3.3057e+00,  1.1

In [8]:
sent2 = "الحباه معفنه" 
idx2 = get_word_idx(sent2, "معفنه")

word_embedding2 = get_word_vector(sent2, idx2, tokenizer, model, layers)

sent3 = "الحباه جميله" 
idx3 = get_word_idx(sent3, "جميله")

word_embedding3 = get_word_vector(sent3, idx3, tokenizer, model, layers)


In [9]:
word_embedding = np.array(word_embedding).reshape(1,-1)
word_embedding2 = np.array(word_embedding2).reshape(1,-1)
word_embedding3 = np.array(word_embedding3).reshape(1,-1)
word_embedding.shape

(1, 768)

* حلوه vs معفنه

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(word_embedding,word_embedding2)

array([[0.55054104]], dtype=float32)

* حلوه vs جميله

In [11]:
cosine_similarity(word_embedding,word_embedding3)


array([[0.46786463]], dtype=float32)

* جميله vs معفنه

In [12]:
cosine_similarity(word_embedding2,word_embedding3)


array([[0.5592834]], dtype=float32)